In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/Users/khairunn/Documents/DAT203/intro-to-dl/Kaggle/final/data"))

# Any results you write to the current directory are saved as output.

/usr/local/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

['test.csv.gz', 'sample_submission.csv.gz', 'shops.csv', 'sales_train.csv.gz', 'item_categories.csv', 'items.csv']


/usr/local/lib/python2.7/site-packages/pandas/core/window.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas._libs.window as _window
/usr/local/lib/python2.7/site-packages/pandas/core/groupby/groupby.py:68: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (lib, reduction,
/usr/local/lib/python2.7/site-packages/pandas/core/reshape/reshape.py:30: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos as _algos, reshape as _reshape
/usr/local/lib/python2.7/site-packages/pandas/io/parsers.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas._libs.parsers as parsers
/usr/local/lib/python2.7/site-packages/pandas/io/pytables.py:50: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibi

# Overall Goals
1.  Feature pre-processing and generation
2. Feature extraction from text and images
3. Advanced Features Generation: mean encodings, PCA, tMNIST, KNN 
4. EDA
5. Validation
6. Hyperparameter Tuning - Primary Layers: KNN, linear models, RF, GBDT, NN
7. Ensembling: Secondary Layers: Single Layer NN, Simple Tree, SVM


## 1. Feature Pre-Processing
### Basic Feature Extraction from Text of Shop Names, Item Names and Item Categories.
- Adding categories to items
- Finding the length of item names and shop names
- Using tfidf to find additional features.


In [2]:
DATA_FOLDER = "/Users/khairunn/Documents/DAT203/intro-to-dl/Kaggle/final/data"

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

In [3]:
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [6]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [7]:
items_and_categories = pd.merge(items, item_categories,on='item_category_id')

In [8]:
items_and_categories.head()

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
2,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
3,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD
4,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40,Кино - DVD


In [9]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


## 2. Feature Extraction
  - Use length to extract features from words
  - Use tfidf to extract features from words

In [10]:
items_and_categories['item_name_len']      = items_and_categories['item_name'].map(len)
items_and_categories['item_category_name'] = items_and_categories['item_category_name'].map(len)
shops['shop_name_len']                     = shops['shop_name'].map(len)

In [11]:
from sklearn import *


feature_cnt = 10
tfidf = feature_extraction.text.TfidfVectorizer(max_features=feature_cnt)

txtFeatures = pd.DataFrame(tfidf.fit_transform(items_and_categories['item_name']).toarray())
cols = txtFeatures.columns
for i in range(feature_cnt):
    items_and_categories['item_name_tfidf_' + str(i)] = txtFeatures[cols[i]]


/usr/local/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/usr/local/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/usr/local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/usr/local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:169: R

/usr/local/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


### Basic Datetime feature generation
- Aggregate by month, by days can probably be used as a feature. 

In [12]:
transactions['date'] = pd.to_datetime(transactions['date'], format='%d.%m.%Y')

transactions['day']   = transactions['date'].dt.day
transactions['month'] = transactions['date'].dt.month
transactions['year']  = transactions['date'].dt.year


Looking at the daily stats

In [13]:
transactions = transactions.groupby([c for c in transactions.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
transactions = transactions.rename(columns={'item_cnt_day':'item_cnt_month'})

In [14]:
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,day,month,year,item_cnt_month
0,2013-01-01,0,2,991,99.0,1,1,2013,1.0
1,2013-01-01,0,2,1472,2599.0,1,1,2013,1.0
2,2013-01-01,0,2,1905,249.0,1,1,2013,1.0
3,2013-01-01,0,2,2920,599.0,1,1,2013,2.0
4,2013-01-01,0,2,3320,1999.0,1,1,2013,1.0


In [15]:
transactions['item_cost_month'] = transactions['item_price'] * transactions['item_cnt_month']

In [16]:
shop_item_monthly_mean = transactions[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_monthly_mean = shop_item_monthly_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})

shop_cost_monthly_mean = transactions[['shop_id','item_id','item_cost_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cost_month']].mean()
shop_cost_monthly_mean = shop_cost_monthly_mean.rename(columns={'item_cost_month':'item_cost_month_mean'})


In [17]:
shop_item_monthly_mean.head()

,shop_id,item_id,item_cnt_month_mean
0,0,30,3.444444
1,0,31,1.571429
2,0,32,1.454545
3,0,33,1.000000
4,0,35,1.250000


In [18]:
shop_cost_monthly_mean.head()

,shop_id,item_id,item_cost_month_mean
0,0,30,912.777778
1,0,31,682.000000
2,0,32,321.454545
3,0,33,347.000000
4,0,35,308.750000


In [19]:
shop_item_monthly_std = transactions[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].std()
shop_item_monthly_std = shop_item_monthly_mean.rename(columns={'item_cnt_month_mean':'item_cnt_month_std'})

shop_cost_monthly_std = transactions[['shop_id','item_id','item_cost_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cost_month']].std()
shop_cost_monthly_std = shop_cost_monthly_mean.rename(columns={'item_cost_month_mean':'item_cost_month_std'})

In [20]:
shop_item_monthly_std.head()

,shop_id,item_id,item_cnt_month_std
0,0,30,3.444444
1,0,31,1.571429
2,0,32,1.454545
3,0,33,1.000000
4,0,35,1.250000


In [21]:
shop_cost_monthly_std.head()

,shop_id,item_id,item_cost_month_std
0,0,30,912.777778
1,0,31,682.000000
2,0,32,321.454545
3,0,33,347.000000
4,0,35,308.750000


In [22]:
transactions = pd.merge(transactions, shop_item_monthly_mean, how='left', on=['shop_id','item_id'])
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,day,month,year,item_cnt_month,item_cost_month,item_cnt_month_mean
0,2013-01-01,0,2,991,99.0,1,1,2013,1.0,99.0,1.000000
1,2013-01-01,0,2,1472,2599.0,1,1,2013,1.0,2599.0,1.000000
2,2013-01-01,0,2,1905,249.0,1,1,2013,1.0,249.0,1.014925
3,2013-01-01,0,2,2920,599.0,1,1,2013,2.0,1198.0,1.000000
4,2013-01-01,0,2,3320,1999.0,1,1,2013,1.0,1999.0,1.400000


In [23]:
# use the last month as validation
shop_item_prev_month = transactions[transactions['date_block_num']==33][['shop_id','item_id','item_cnt_month']]
shop_item_prev_month = shop_item_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})
shop_item_prev_month.head()


,shop_id,item_id,item_cnt_prev_month
2882329,2,2791,1.0
2882330,2,3351,1.0
2882331,2,3353,1.0
2882332,2,3708,1.0
2882333,2,4388,1.0


Time to merge the datasets

In [24]:
transactions = pd.merge(transactions, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)


In [25]:
transactions = pd.merge(transactions, items_and_categories, how='left', on='item_id')


In [26]:
transactions = pd.merge(transactions, shops, how='left', on='shop_id')

In [27]:
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,day,month,year,item_cnt_month,item_cost_month,...,item_name_tfidf_2,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,item_name_tfidf_6,item_name_tfidf_7,item_name_tfidf_8,item_name_tfidf_9,shop_name,shop_name_len
0,2013-01-01,0,2,991,99.0,1,1,2013,1.0,99.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,"Адыгея ТЦ ""Мега""",28
1,2013-01-01,0,2,1472,2599.0,1,1,2013,1.0,2599.0,...,0.0,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Адыгея ТЦ ""Мега""",28
2,2013-01-01,0,2,1905,249.0,1,1,2013,1.0,249.0,...,0.0,0.772083,0.635522,0.0,0.000000,0.0,0.000000,0.0,"Адыгея ТЦ ""Мега""",28
3,2013-01-01,0,2,2920,599.0,1,1,2013,2.0,1198.0,...,0.0,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Адыгея ТЦ ""Мега""",28
4,2013-01-01,0,2,3320,1999.0,1,1,2013,1.0,1999.0,...,0.0,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Адыгея ТЦ ""Мега""",28


Making adjustments to the test set
1. Adding date
2. Adding items, categories and shops

In [28]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34

In [29]:
test = pd.merge(test, shop_item_monthly_mean, how='left', on=['shop_id','item_id']).fillna(0.)

In [30]:
test = pd.merge(test, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)

In [31]:
test = pd.merge(test, items_and_categories, how='left', on='item_id')

In [32]:
test = pd.merge(test, shops, how='left', on='shop_id')

In [33]:
test['item_cnt_month'] = 0

In [34]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,...,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,item_name_tfidf_6,item_name_tfidf_7,item_name_tfidf_8,item_name_tfidf_9,shop_name,shop_name_len,item_cnt_month
0,0,5,5037,11,2015,34,1.000000,0.0,"NHL 15 [PS3, русские субтитры]",19,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
1,1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,...,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
2,2,5,5233,11,2015,34,1.111111,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,...,0.0,0.0,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0
3,3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,...,0.0,0.0,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0
4,4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,...,0.0,0.0,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0


## 3. Advanced Feature Generation
- Encoding
- Use pca
- Use tMNIST
- Use kNN

### Encoding

In [35]:
print(test.shape)
print(transactions.shape)

feat_cols = [c for c in test.columns if c in transactions.columns]
print(len(feat_cols))

(235052, 25)
(4480317, 28)
24


In [36]:
test_feat = test[feat_cols]
y_train = transactions['item_cnt_month']
transactions_feat_cols = transactions[feat_cols]

In [37]:
test_feat.head()

,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,...,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,item_name_tfidf_6,item_name_tfidf_7,item_name_tfidf_8,item_name_tfidf_9,shop_name,shop_name_len,item_cnt_month
0,5,5037,11,2015,34,1.000000,0.0,"NHL 15 [PS3, русские субтитры]",19,14,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,63,...,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
2,5,5233,11,2015,34,1.111111,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,14,...,0.0,0.0,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0
3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,19,...,0.0,0.0,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0
4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,14,...,0.0,0.0,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0


In [38]:
transactions_feat_cols.head()

,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,...,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,item_name_tfidf_6,item_name_tfidf_7,item_name_tfidf_8,item_name_tfidf_9,shop_name,shop_name_len,item_cnt_month
0,2,991,1,2013,0,1.000000,0.0,"3D Action Puzzle ""Динозавры"" Тиранозавр",67,33,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,"Адыгея ТЦ ""Мега""",28,1.0
1,2,1472,1,2013,0,1.000000,0.0,"Assassin's Creed 3 [Xbox 360, русская версия]",23,19,...,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Адыгея ТЦ ""Мега""",28,1.0
2,2,1905,1,2013,0,1.014925,0.0,"Bestseller. Grand Theft Auto: San Andreas [PC,...",30,51,...,0.772083,0.635522,0.0,0.000000,0.0,0.000000,0.0,"Адыгея ТЦ ""Мега""",28,1.0
3,2,2920,1,2013,0,1.000000,0.0,Disney. LEGO Пираты Карибского моря (Essential...,21,14,...,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Адыгея ТЦ ""Мега""",28,2.0
4,2,3320,1,2013,0,1.400000,0.0,"FIFA 13 (с поддержкой PS Move) [PS3, русская в...",19,14,...,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Адыгея ТЦ ""Мега""",28,1.0


In [39]:
transactions_feat_cols.drop('item_name', axis =1)
test_feat.drop('item_name', axis =1)

,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_category_id,item_category_name,item_name_len,...,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,item_name_tfidf_6,item_name_tfidf_7,item_name_tfidf_8,item_name_tfidf_9,shop_name,shop_name_len,item_cnt_month
0,5,5037,11,2015,34,1.000000,0.0,19,14,45,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
1,5,5320,11,2015,34,0.000000,0.0,55,63,31,...,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
2,5,5233,11,2015,34,1.111111,1.0,19,14,69,...,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0
3,5,5232,11,2015,34,1.000000,0.0,23,19,72,...,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0
4,5,5268,11,2015,34,0.000000,0.0,20,14,49,...,0.000000,0.000000,0.0,0.606187,0.0,0.795322,0.0,"Вологда ТРЦ ""Мармелад""",40,0
5,5,5039,11,2015,34,1.000000,1.0,23,19,50,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
6,5,5041,11,2015,34,1.000000,1.0,20,14,45,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
7,5,5041,11,2015,34,1.000000,1.0,20,14,45,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
8,5,5046,11,2015,34,1.000000,0.0,55,63,28,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0
9,5,5319,11,2015,34,1.340426,0.0,55,63,19,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",40,0


In [40]:
transactions_feat_cols = transactions_feat_cols.drop(['shop_name','item_name'], axis =1)
test_feat = test_feat.drop(['shop_name','item_name'], axis =1)

In [41]:
from sklearn.decomposition import PCA

In [42]:
def pca_find_dimensions(dimensions):
    pca = PCA(n_components=dimensions)
    pca_result = pca.fit_transform(transactions_feat_cols.values)
    
    print ('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [43]:
for i in range(5):
    pca_find_dimensions(i)

Explained variation per principal component: []
Explained variation per principal component: [0.99994478]
Explained variation per principal component: [9.99944785e-01 2.04534722e-05]
Explained variation per principal component: [9.99944785e-01 2.04534722e-05 1.10826960e-05]
Explained variation per principal component: [9.99944785e-01 2.04534722e-05 1.10826960e-05 1.03189336e-05]


In [45]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(transactions_feat_cols.values)

In [46]:
transactions_feat_cols['pca_result_0'] = pca_result[:,0]
transactions_feat_cols['pca_result_1'] = pca_result[:,1]

In [47]:
test_feat_pca = pca.transform(test_feat)

In [48]:
test_feat['pca_result_0'] = test_feat_pca[:,0]
test_feat['pca_result_1'] = test_feat_pca[:,1]

### tMNIST

In [ ]:
rndperm = np.random.permutation(transactions_feat_cols.shape[0])

from sklearn.manifold import TSNE

n_sne = 7000

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(transactions_feat_cols.loc[rndperm[:n_sne],].values)


In [52]:
transactions_feat_cols = df.loc[rndperm[:n_sne],:].copy()
df_tsne['x-tsne'] = tsne_results[:,0]
df_tsne['y-tsne'] = tsne_results[:,1]

AttributeError: 'TSNE' object has no attribute 'transform'